In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
import pandas as pd
import time
import re
from datetime import datetime
import os


def setup_driver():
  
    options = webdriver.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--window-size=1920,1080")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36")

    driver = webdriver.Chrome(options=options)
    return driver


def scroll_to_load_jobs(driver, target_jobs=200, scroll_pause_time=1.5):
    
    print(f"  Scrolling to load up to {target_jobs} jobs...")

    last_height = driver.execute_script("return document.body.scrollHeight")
    jobs_loaded = 0
    scroll_attempts = 0
    max_attempts = 20
    consecutive_no_new_jobs = 0

    while jobs_loaded < target_jobs and scroll_attempts < max_attempts:
        scroll_attempts += 1

        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)

        
        time.sleep(0.5)

       
        try:
            current_jobs = driver.find_elements(By.CSS_SELECTOR, "tr.job")
            new_jobs_loaded = len(current_jobs)
        except StaleElementReferenceException:
            current_jobs = driver.find_elements(By.CSS_SELECTOR, "tr.job")
            new_jobs_loaded = len(current_jobs)

        
        if new_jobs_loaded > jobs_loaded:
            jobs_loaded = new_jobs_loaded
            consecutive_no_new_jobs = 0
            if jobs_loaded % 50 == 0:
                print(f"    Loaded {jobs_loaded} jobs...")
        else:
            consecutive_no_new_jobs += 1

       
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            if consecutive_no_new_jobs > 2:
                print(f"    Reached bottom. Found {jobs_loaded} jobs.")
                break
        else:
            last_height = new_height

        
        if jobs_loaded >= target_jobs:
            break

    print(f"    Total loaded from this page: {jobs_loaded} jobs")
    return jobs_loaded


def extract_location_details(location_text):
    
    country = ""
    city = ""

    if not location_text or location_text.lower() in ['remote', 'anywhere', 'global', 'worldwide']:
        country = "Remote"
        city = "Remote"
    else:
        location_text = location_text.strip()

       
        timezone_patterns = ['est', 'pst', 'cst', 'mst', 'gmt', 'utc']
        for tz in timezone_patterns:
            if tz in location_text.lower():
                country = f"{tz.upper()} Timezone"
                city = location_text
                return country, city

        
        countries = ['usa', 'united states', 'us', 'uk', 'united kingdom', 'canada',
                     'germany', 'france', 'spain', 'italy', 'india', 'australia',
                     'netherlands', 'sweden', 'denmark', 'norway', 'finland',
                     'brazil', 'mexico', 'argentina', 'singapore', 'japan']

        for c in countries:
            if c in location_text.lower():
                country = c.title()
                parts = re.split(r'[,|\-]', location_text, flags=re.IGNORECASE)
                if len(parts) > 1:
                    city = parts[0].strip()
                else:
                    city = re.sub(c, '', location_text, flags=re.IGNORECASE).strip()
                    if not city:
                        city = "Multiple locations"
                break

        if not country:
            country = location_text
            city = "Multiple locations"

    return country, city


def categorize_job(title, tags):
    
    title_lower = title.lower()
    tags_lower = ' '.join([tag.lower() for tag in tags]) if tags else ""

    categories = {
        'Data Analysis': ['data analyst', 'data analysis', 'analytics', 'bi analyst',
                          'business intelligence', 'sql', 'tableau', 'power bi', 'excel'],
        'Data Science': ['data scientist', 'data science', 'machine learning', 'ml',
                         'ai', 'artificial intelligence', 'deep learning', 'nlp'],
        'Software Development': ['software engineer', 'developer', 'programmer',
                                 'full stack', 'frontend', 'backend', 'web developer'],
        'DevOps': ['devops', 'sre', 'site reliability', 'cloud engineer', 'aws',
                   'azure', 'docker', 'kubernetes'],
        'Design': ['designer', 'ui/ux', 'ui ', 'ux ', 'graphic design', 'product design'],
        'Marketing': ['marketing', 'seo', 'sem', 'digital marketing', 'growth hacker'],
        'Sales': ['sales', 'account executive', 'business development', 'bd'],
        'Support': ['support', 'customer success', 'help desk', 'technical support'],
        'Product Management': ['product manager', 'product owner', 'pm ', 'product management'],
        'Finance': ['finance', 'accounting', 'financial analyst', 'cpa', 'accountant'],
        'QA/Testing': ['qa', 'quality assurance', 'test engineer', 'automation testing'],
        'Project Management': ['project manager', 'scrum master', 'agile'],
        'Content Writing': ['content writer', 'copywriter', 'technical writer', 'blogger']
    }

    for category, keywords in categories.items():
        for keyword in keywords:
            if keyword in title_lower or keyword in tags_lower:
                return category

    return 'Other'


def extract_skills_from_tags(tags):
    
    if not tags:
        return []

    skill_categories = {
        'Programming Languages': [
            'python', 'javascript', 'java', 'typescript', 'c#', 'c++', 'ruby',
            'php', 'go', 'rust', 'swift', 'kotlin', 'scala', 'r', 'matlab'
        ],
        'Frontend Technologies': [
            'react', 'angular', 'vue', 'next.js', 'nuxt.js', 'svelte',
            'html', 'css', 'sass', 'less', 'tailwind', 'bootstrap', 'jquery'
        ],
        'Backend Frameworks': [
            'node.js', 'django', 'flask', 'spring', 'express.js', 'laravel',
            'rails', 'asp.net', 'fastapi', 'nestjs', 'graphql'
        ],
        'Databases': [
            'sql', 'mysql', 'postgresql', 'mongodb', 'redis', 'elasticsearch',
            'oracle', 'sql server', 'cassandra', 'dynamodb', 'firebase'
        ],
        'Cloud & DevOps': [
            'aws', 'azure', 'gcp', 'docker', 'kubernetes', 'terraform',
            'ansible', 'jenkins', 'ci/cd', 'github actions', 'gitlab ci'
        ],
        'Data Science & ML': [
            'pandas', 'numpy', 'scikit-learn', 'tensorflow', 'pytorch', 'keras',
            'spark', 'hadoop', 'hive', 'airflow', 'tableau', 'power bi', 'looker'
        ],
        'Tools & Platforms': [
            'git', 'github', 'gitlab', 'jira', 'confluence', 'figma', 'slack',
            'trello', 'asana', 'notion', 'wordpress', 'shopify', 'salesforce'
        ]
    }

    tags_lower = [tag.lower() for tag in tags]
    skills_found = []

    for category, skill_list in skill_categories.items():
        for skill in skill_list:
            for tag in tags_lower:
                skill_variations = [skill]
                if skill.endswith('.js'):
                    skill_variations.append(skill.replace('.js', ''))

                for skill_var in skill_variations:
                    if skill_var in tag:
                        formatted_skill = skill.title() if '.' not in skill else skill
                        skills_found.append(formatted_skill)
                        break

    for tag in tags_lower:
        if any(lang in tag for lang in ['english', 'spanish', 'french', 'german', 'chinese']):
            skills_found.append(tag.title())

    skills_found = list(dict.fromkeys(skills_found))
    skills_found.sort()

    return skills_found


def extract_job_details(job_row, driver):
   
    try:
        title_elem = job_row.find_element(By.CSS_SELECTOR, "h2[itemprop='title']")
        title = title_elem.text.strip() if title_elem else ""

        company_elem = job_row.find_element(By.CSS_SELECTOR, "h3[itemprop='name']")
        company = company_elem.text.strip() if company_elem else ""

        link_elem = job_row.find_element(By.CSS_SELECTOR, "a[itemprop='url']")
        job_url = link_elem.get_attribute("href") if link_elem else ""

        tags = []
        try:
            tag_elems = job_row.find_elements(By.CSS_SELECTOR, "td.tags h3, td.tags div.tag")
            for tag_elem in tag_elems:
                tag_text = tag_elem.text.strip()
                if tag_text:
                    tags.append(tag_text)
        except:
            pass

        salary = ""
        try:
            salary_elem = job_row.find_elements(By.CSS_SELECTOR, ".location, .salary")
            for elem in salary_elem:
                text = elem.text.strip()
                if "$" in text or "€" in text or "£" in text or "salary" in text.lower():
                    salary = text
                    break
        except:
            pass

        date_posted = ""
        try:
            date_elem = job_row.find_element(By.CSS_SELECTOR, "time")
            if date_elem:
                date_posted = date_elem.get_attribute("datetime") or date_elem.text.strip()
        except:
            pass

        location_text = "Remote"
        try:
            location_selectors = [
                ".location:not(:has(*))",
                ".location span",
                "td.location",
                "div.location"
            ]

            for selector in location_selectors:
                try:
                    loc_elems = job_row.find_elements(By.CSS_SELECTOR, selector)
                    for elem in loc_elems:
                        text = elem.text.strip()
                        if (text and
                                "$" not in text and
                                "€" not in text and
                                "£" not in text and
                                "salary" not in text.lower()):
                            location_text = text
                            break
                except:
                    continue
                if location_text != "Remote":
                    break
        except:
            pass

        country, city = extract_location_details(location_text)
        category = categorize_job(title, tags)
        skills_list = extract_skills_from_tags(tags)

        job_record = {
            "Job Title": title,
            "Company Name": company,
            "Job URL": job_url,
            "Job Tags": ", ".join(tags) if tags else "",
            "Salary Range": salary,
            "Date Posted": date_posted,
            "Location": location_text,
            "Country": country,
            "City": city,
            "Job Category": category,
            "Skills Found": ", ".join(skills_list) if skills_list else "None identified",
            "Skills Count": len(skills_list),
            "Key Skill 1": skills_list[0] if len(skills_list) > 0 else "",
            "Key Skill 2": skills_list[1] if len(skills_list) > 1 else "",
            "Key Skill 3": skills_list[2] if len(skills_list) > 2 else "",
            "Key Skill 4": skills_list[3] if len(skills_list) > 3 else "",
            "Key Skill 5": skills_list[4] if len(skills_list) > 4 else ""
        }

        return job_record

    except Exception as e:
        return None


def scrape_single_page(driver, url, target_jobs=200):
    
    print(f"  Loading: {url}")

    try:
        driver.get(url)
        time.sleep(3)

        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "tr.job"))
        )
    except TimeoutException:
        print(f"    No jobs found on {url}")
        return []

    scroll_to_load_jobs(driver, target_jobs)

    try:
        job_rows = driver.find_elements(By.CSS_SELECTOR, "tr.job")
    except:
        print("    Could not find job rows")
        return []

    print(f"    Processing {len(job_rows)} jobs...")

    page_jobs = []
    for i, job_row in enumerate(job_rows, 1):
        try:
            job_data = extract_job_details(job_row, driver)
            if job_data:
                page_jobs.append(job_data)

                if i % 50 == 0:
                    print(f"      Processed {i} jobs...")

        except Exception:
            continue

    print(f"    Extracted {len(page_jobs)} jobs from this page")
    return page_jobs


def scrape_multiple_categories(target_jobs=700):

    driver = None
    all_jobs = []
    jobs_collected = 0


    categories = [
        ("main", "https://remoteok.com"),
        ("development", "https://remoteok.com/remote-dev-jobs"),
        ("design", "https://remoteok.com/remote-design-jobs"),
        ("marketing", "https://remoteok.com/remote-marketing-jobs"),
        ("sales", "https://remoteok.com/remote-sales-jobs"),
        ("product", "https://remoteok.com/remote-product-jobs"),
        ("support", "https://remoteok.com/remote-customer-support-jobs"),
        ("finance", "https://remoteok.com/remote-finance-jobs"),
        ("writing", "https://remoteok.com/remote-writing-jobs"),
        ("legal", "https://remoteok.com/remote-legal-jobs"),
        ("all-jobs", "https://remoteok.com/remote-jobs")
    ]

    try:
        driver = setup_driver()

        print(f"\nStarting comprehensive scraping for {target_jobs} jobs...")
        print("This will scrape multiple job categories.")
        print("Please wait while it processes each category...\n")

        start_time = time.time()

        for category_name, url in categories:
            if jobs_collected >= target_jobs:
                print(f"\nTarget reached! Collected {jobs_collected} jobs.")
                break

            print(f"\n[{len(all_jobs) + 1}] Scraping {category_name.upper()} category...")


            category_jobs = scrape_single_page(driver, url, target_jobs=150)

            if category_jobs:

                existing_urls = {job['Job URL'] for job in all_jobs}
                new_jobs = []

                for job in category_jobs:
                    if job['Job URL'] not in existing_urls:
                        new_jobs.append(job)
                        existing_urls.add(job['Job URL'])

                all_jobs.extend(new_jobs)
                jobs_collected = len(all_jobs)

                print(f"  Added {len(new_jobs)} new jobs from {category_name}")
                print(f"  Total so far: {jobs_collected} jobs")


            if jobs_collected < target_jobs:
                time.sleep(2)


        if jobs_collected < target_jobs:
            print("\n\nTrying search queries to get more jobs...")

            search_queries = [
                "python", "javascript", "react", "aws", "devops",
                "data", "analyst", "engineer", "developer", "remote"
            ]

            for query in search_queries:
                if jobs_collected >= target_jobs:
                    break

                print(f"\nSearching for: '{query}'")
                search_url = f"https://remoteok.com/?search={query}"

                search_jobs = scrape_single_page(driver, search_url, target_jobs=100)

                if search_jobs:
                    existing_urls = {job['Job URL'] for job in all_jobs}
                    new_jobs = []

                    for job in search_jobs:
                        if job['Job URL'] not in existing_urls:
                            new_jobs.append(job)
                            existing_urls.add(job['Job URL'])

                    all_jobs.extend(new_jobs)
                    jobs_collected = len(all_jobs)

                    print(f"  Added {len(new_jobs)} new jobs from '{query}' search")
                    print(f"  Total so far: {jobs_collected} jobs")

                time.sleep(2)

        elapsed_time = time.time() - start_time
        minutes = int(elapsed_time // 60)
        seconds = int(elapsed_time % 60)

        print(f"\n{'=' * 60}")
        print(f"COMPREHENSIVE SCRAPING COMPLETE")
        print(f"{'=' * 60}")
        print(f"Time taken: {minutes} minutes {seconds} seconds")
        print(f"Target jobs: {target_jobs}")
        print(f"Jobs collected: {jobs_collected}")

        if jobs_collected < target_jobs:
            print(f"\nNote: Could only collect {jobs_collected} jobs.")
            print("RemoteOK may be limiting the number of visible jobs.")
            print("Try running again later or use the archived jobs feature.")

    except Exception as e:
        print(f"Error during comprehensive scraping: {e}")

    finally:
        if driver:
            driver.quit()

    return all_jobs


def save_data_to_excel(jobs_data, filename="remoteok_700plus_jobs.xlsx"):

    if not jobs_data:
        print("No jobs data to save.")
        return None

    df = pd.DataFrame(jobs_data)


    initial_count = len(df)
    df.drop_duplicates(subset=["Job URL"], inplace=True, keep='first')
    final_count = len(df)

    if initial_count != final_count:
        print(f"Removed {initial_count - final_count} duplicate jobs")

    print(f"\nSaving {len(df)} jobs to Excel...")


    with pd.ExcelWriter(filename, engine='openpyxl') as writer:

        df.to_excel(writer, sheet_name='All Jobs', index=False)


        print("Creating summary sheets...")


        if 'Skills Found' in df.columns:
            all_skills = []
            for skills_str in df['Skills Found']:
                if skills_str and skills_str != "None identified":
                    skills = [s.strip() for s in skills_str.split(',')]
                    all_skills.extend(skills)

            if all_skills:
                skills_series = pd.Series(all_skills)
                skills_summary = skills_series.value_counts().reset_index()
                skills_summary.columns = ['Skill Name', 'Job Count']
                skills_summary['Percentage'] = (skills_summary['Job Count'] / len(df) * 100).round(2)
                skills_summary.to_excel(writer, sheet_name='Skills Analysis', index=False)


        if 'Job Category' in df.columns:
            category_summary = df['Job Category'].value_counts().reset_index()
            category_summary.columns = ['Job Category', 'Count']
            category_summary['Percentage'] = (category_summary['Count'] / len(df) * 100).round(2)
            category_summary.to_excel(writer, sheet_name='By Category', index=False)


        if 'Country' in df.columns:
            country_summary = df['Country'].value_counts().reset_index()
            country_summary.columns = ['Country', 'Count']
            country_summary['Percentage'] = (country_summary['Count'] / len(df) * 100).round(2)
            country_summary.to_excel(writer, sheet_name='By Country', index=False)


        if 'Skills Count' in df.columns:
            high_skill_df = df[df['Skills Count'] >= 3].copy()
            high_skill_df = high_skill_df.sort_values('Skills Count', ascending=False)
            high_skill_df.to_excel(writer, sheet_name='High Skill Jobs', index=False)


        if 'Date Posted' in df.columns:
            try:
                df['Date Posted'] = pd.to_datetime(df['Date Posted'], errors='coerce')
                recent_df = df[df['Date Posted'] >= pd.Timestamp.now() - pd.Timedelta(days=30)]
                if not recent_df.empty:
                    recent_df.to_excel(writer, sheet_name='Recent Jobs', index=False)
            except:
                pass


        if 'Company Name' in df.columns:
            company_summary = df['Company Name'].value_counts().reset_index()
            company_summary.columns = ['Company Name', 'Job Count']
            company_summary = company_summary.head(20)  # Top 20 companies
            company_summary.to_excel(writer, sheet_name='Top Companies', index=False)

    print(f"\nExcel file created: {filename}")
    print(f"File size: {os.path.getsize(filename) / 1024 / 1024:.2f} MB")
    print(f"File location: {os.path.abspath(filename)}")

    return df


def main():
    print("=" * 80)
    print("REMOTEOK 700+ JOBS SCRAPER - MULTI-CATEGORY APPROACH")
    print("=" * 80)

    target_jobs = 700

    print(f"\nThis scraper will collect jobs from multiple categories")
    print(f"to reach the target of {target_jobs} jobs.")
    print("\nEstimated time: 10-15 minutes")
    print("A Chrome browser will open and automatically navigate through categories.")

    input("\nPress Enter to start scraping...")


    jobs_data = scrape_multiple_categories(target_jobs)

    if jobs_data:

        filename = "remoteok_700plus_jobs.xlsx"
        df = save_data_to_excel(jobs_data, filename)


        print("\n" + "=" * 80)
        print("FINAL SUMMARY")
        print("=" * 80)
        print(f"Total unique jobs collected: {len(df)}")

        if len(df) > 0:

            print("\nKEY STATISTICS:")

            if 'Job Category' in df.columns:
                print("\nTop Job Categories:")
                top_categories = df['Job Category'].value_counts().head(5)
                for category, count in top_categories.items():
                    percentage = (count / len(df)) * 100
                    print(f"  - {category:<25} {count:>4} jobs ({percentage:.1f}%)")

            if 'Country' in df.columns:
                print("\nTop Countries:")
                top_countries = df['Country'].value_counts().head(5)
                for country, count in top_countries.items():
                    percentage = (count / len(df)) * 100
                    print(f"  - {country:<25} {count:>4} jobs ({percentage:.1f}%)")

            if 'Skills Found' in df.columns:
                jobs_with_skills = df[df['Skills Found'] != "None identified"].shape[0]
                print(f"\nJobs with identified skills: {jobs_with_skills} ({jobs_with_skills / len(df) * 100:.1f}%)")


                all_skills = []
                for skills_str in df['Skills Found']:
                    if skills_str and skills_str != "None identified":
                        skills = [s.strip() for s in skills_str.split(',')]
                        all_skills.extend(skills)

                if all_skills:
                    top_skills = pd.Series(all_skills).value_counts().head(5)
                    print("\nTop 5 Most Demanded Skills:")
                    for skill, count in top_skills.items():
                        percentage = (count / len(df)) * 100
                        print(f"  - {skill:<20} {count:>4} jobs ({percentage:.1f}%)")


            print("\nSAMPLE JOBS (first 3):")
            for i, (idx, row) in enumerate(df.head(3).iterrows(), 1):
                print(f"\n{i}. {row.get('Job Title', 'N/A')}")
                print(f"   Company: {row.get('Company Name', 'N/A')}")
                print(f"   Category: {row.get('Job Category', 'N/A')}")
                print(f"   Country: {row.get('Country', 'N/A')}")
                skills = row.get('Skills Found', '')
                if skills and skills != "None identified":
                    print(f"   Skills: {skills[:60]}..." if len(skills) > 60 else f"   Skills: {skills}")

        print(f"\n{'=' * 80}")
        print("NEXT STEPS:")
        print("1. Open the Excel file: remoteok_700plus_jobs.xlsx")
        print("2. Use the 'All Jobs' sheet for Power BI import")
        print("3. Use summary sheets for quick analysis")
        print("4. Filter and analyze the data as needed")

    else:
        print("\nNo jobs were collected. Possible reasons:")
        print("1. Internet connection issue")
        print("2. RemoteOK website structure changed")
        print("3. Anti-bot measures detected")
        print("\nTry running again or check your internet connection.")


if __name__ == "__main__":
    main()